In [2]:
import pandas as pd
import numpy as np

C:\Users\enzok\AppData\Local\Temp\ipykernel_48084\2937650959.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df_train = pd.read_csv('for_students/waiting_times_train.csv')

In [9]:
date_time = df_train["DATETIME"]

from datetime import datetime

# Fonction pour convertir la date en une liste avec le jour de la semaine, le mois et l'année
def convert_date(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    jour_semaine = date_obj.strftime('%A')  # Jour de la semaine (nom complet)
    mois = date_obj.strftime('%B')  # Mois (nom complet)
    annee = date_obj.year  # Année
    return [jour_semaine, mois, annee]

times = []
years = []
months = []
days_week = []

for date in date_time:
    day, month, year = convert_date(date[:10])
    years.append(year)
    months.append(month)
    days_week.append(day)
    time = date[11:16]
    times.append(int(time[:2]) * 4 + int(time[3:5]) // 15)

times = np.array(times) - np.min(times)

encoded_years = pd.get_dummies(years)
encoded_months = pd.get_dummies(months)
encoded_days_week = pd.get_dummies(days_week)
encoded_times = pd.get_dummies(times)

print(encoded_days_week)
print(encoded_times)
print(encoded_years)
print(encoded_months)

          0      1      2      3      4      5      6      7      8      9   \
0      False  False  False  False  False  False  False  False  False  False   
1      False  False  False  False  False  False  False   True  False  False   
2      False  False  False  False  False  False  False  False  False  False   
3      False  False  False  False  False  False  False  False  False  False   
4      False  False  False  False  False  False   True  False  False  False   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
37013  False  False  False  False  False  False  False  False  False  False   
37014  False  False  False  False  False  False  False  False  False  False   
37015  False  False  False  False  False  False  False  False  False  False   
37016  False  False  False  False  False  False  False  False  False  False   
37017  False  False  False  False  False   True  False  False  False  False   

       ...     38     39     40     41     42     4